In [ ]:
import os

# Change working dir so that we can import local module
working_dir = os.path.join(os.getcwd(), os.pardir)
os.chdir(working_dir)
from reshpc import nersc_interface as rn

In [ ]:
nersc = rn.NerscInterface()
nersc.login()

In [ ]:
# Generate new folder name yymmdd_hhmmss
import datetime
now = datetime.datetime.now()
datetime_code = now.strftime('%y%m%d-%H%M%S')
datetime_code

In [ ]:
# Generate path on Cori $SCRATCH folder
scratch_folder = nersc.get_scratch_folder()
nersc_folder = '{}/reshpc/{}'.format(scratch_folder, datetime_code)
nersc_folder

In [ ]:
nersc.make_folder(nersc_folder)

In [ ]:
# Upload demo 01 baseline file
xml_file = 'demo_01.xml'
local_file = os.path.join(working_dir, 'data', xml_file)
assert os.path.exists(local_file)

nersc.upload_file(local_file, nersc_folder)

In [ ]:
# Specify SLURM script
ats = '/project/projectdirs/m2398/ideas/ats/install/cori/ats-0.88-basic/RelWithDebInfo/PrgEnv-gnu-6.0.5/bin/ats'
timeout_min = 5
nodes = 1
cores_per_node = 2

slurm_commands = [
    '#!/bin/bash',
    '#SBATCH --chdir={}'.format(nersc_folder),
    '#SBATCH --qos=debug',
    '#SBATCH --time={}'.format(timeout_min),
    '#SBATCH --nodes={}'.format(nodes),
    '#SBATCH --tasks-per-node={}'.format(cores_per_node),
    '#SBATCH --constraint=haswell',
    'srun -C haswell {} --xml_file={}'.format(ats, xml_file)
]
slurm_string = '\n'.join(slurm_commands)

In [ ]:
# Submit job
result = nersc.submit_job(slurm_string, nersc_folder)
print(result)

In [ ]:
job_id = result.get('jobid')
if job_id:
    status = nersc.get_job_status(job_id)
    print(status)


In [ ]:
job_info = nersc.get_job_info(job_id, verbose=True)
print(job_info)
status = job_info.get('status')
print('Status {}'.format(status))

In [ ]:
import time
# Loop while status is pending, running, or completing
while status not in ['PD', 'R', 'CG']:
    time.sleep(5.0)
    info = nersc.get_job_info(job_id)
    status = info.get('status')
    print(status)